# Code

In [9]:
from joblib import Parallel, delayed
import xgboost as xgb
from sklearn.metrics import mean_squared_error
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import numpy as np 
from os import path
import pandas as pd 
import os
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
os.environ["CUDA_VISIBLE_DEVICES"] = '0'
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
from matplotlib import rc
from matplotlib import rcParams
import datetime
import math
import time
import pickle
import random
from scipy.spatial import cKDTree
from sklearn import preprocessing
from sklearn.decomposition import PCA
import sys
from sklearn.metrics import r2_score as r2
from rdkit import Chem
from sklearn.decomposition import PCA

import importlib
import sys
sys.path.append('/home/rgur/py_scripts')
import efrc_ml_production as ml
importlib.reload(ml)

from skopt import gp_minimize

from multiprocessing import Pool

import hmofMLdataset as hmof
importlib.reload(hmof)

<module 'hmofMLdataset' from '/home/rgur/py_scripts/hmofMLdataset.py'>

# Run

In [10]:
PATH = '/data/rgur/efrc/ml/tests/specify_jobs'

In [11]:
JOB_DICT = {'10000': ([.5], [20])}

In [ ]:
now = datetime.datetime.now().strftime("%I_%M%p_on_%B_%d_%Y")

In [12]:
hmof.hmofMLdataset(PATH, now,
                   SI_grav_data_path='/data/rgur/efrc/prep_data/all_v1/ml_data_head.csv', 
                 SD_grav_data_path='/data/rgur/efrc/prep_data/all_no_norm/ml_data_head.csv',SI_stacked_path=
                '/data/rgur/efrc/prep_data/all_v1/stacked_head.csv',
                 SD_stacked_path='/data/rgur/efrc/prep_data/all_no_norm/stacked_head.csv', n_core=15,
             job_dict=JOB_DICT).makeAllResults()

There are 1 unique feature codes
now is 04_39PM_on_April_22_2020

Starting to Construct Gravimetric Uptake Data Frame
Using start_str_sd CH4_v/v_248_bar
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


Starting To Make Linker Size Columns
Starting to sort Linker Size Columns
Finished Making Linker Size Columns
The following columns have been dropped: ['norm_Mafp_C1_N2_N3', 'norm_Mafp_N2_O2_N3', 'norm_Mmfp_MQNs22', 'norm_Mmfp_MQNs23', 'norm_Mmfp_MQNs24', 'norm_Mmfp_MQNs25']

Starting to Construct Isotherm Stacked Data Frame
Using start_str_sd Density
Using end_str_sd norm_Dom._Pore_(ang.)
Using start_str_si filename
Using end_str_si valence_pa
Total frac equals 1


Starting To Make Linker Size Columns
Finished Making Linker Size Columns
The following columns have been dropped: ['norm_Mafp_C1_N2_N3', 'norm_Mafp_N2_O2_N3', 'norm_Mmfp_MQNs22', 'norm_Mmfp_MQNs23', 'norm_Mmfp_MQNs24', 'norm_Mmfp_MQNs25']


Running code 1000 